In [1]:
import matplotlib.pyplot as plt
%matplotlib
import numpy as np
import pandas as pd
import scipy
import math
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D

from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor

from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import validation_curve
from sklearn.model_selection import GridSearchCV

Using matplotlib backend: MacOSX


### kNN
- 

In [ ]:
#read in used car CSV file
df = pd.read_csv('/Users/tk/ASU/STP494/data/susedcars.csv')